Ronit Kumar Kataria
rk06451@st.habib.edu.pk


# Part 1
Data Cleaning

In [51]:
import pandas as pd

In [52]:
df = pd.read_csv('580SurveyCleanup.csv')

In [53]:
df_1 = df.iloc[:,1:12]
df_1.columns = df_1.iloc[0]
df_1 = df_1[1:]



In [54]:
question_dictonary = {
    1: [12, 13],
    2: [14, 17],
    3: [18, 18],
    4: [19, 19],
    5: [20, 20],
    6: [21, 21],
    7: [22, 22],
    8: [23, 26],
    9: [27, 29],
    10: [30, 34],
    11: [35, 39],
    12: [40, 44],
    13: [45, 45]
}


In [55]:
def remove_null_entries(start, end):
  temp = []
  for i in range(start, end+1):
    temp.append(i)
  df1 = df.iloc[:,temp]
  # Convert all null values to 0
  df1.columns = df1.iloc[0].astype(str)
  df1 = df1[1:]
  column_list = []
  for i in df1.columns:
    column_list.append(i)
  df2 = df1.loc[:, column_list].replace("1", pd.Series(df1.columns, df1.columns)).fillna('')
  return df2

In [56]:
data_lst = []
for i in question_dictonary:
  temp = remove_null_entries(question_dictonary[i][0], question_dictonary[i][1])
  data_lst.append(temp)  

In [57]:
replaced_data = [None] * len(data_lst)
for i in range(len(replaced_data)):
    column_lst = list(data_lst[i].columns)
    replaced_data[i] = data_lst[i].loc[:, column_lst].replace(
        1, pd.Series(data_lst[i].columns, data_lst[i].columns)).fillna('')
  

Combining All Columns together

In [58]:
replaced_data[0] = pd.DataFrame(replaced_data[0].iloc[:,0] + replaced_data[0].iloc[:,1])
replaced_data[1] = pd.DataFrame(replaced_data[1].iloc[:,0] + replaced_data[1].iloc[:,1] + replaced_data[1].iloc[:,2]+ replaced_data[1].iloc[:,3])
replaced_data[2] = pd.DataFrame(replaced_data[2].iloc[:,0])
replaced_data[3] = pd.DataFrame(replaced_data[3].iloc[:,0])
replaced_data[4] = pd.DataFrame(replaced_data[4].iloc[:,0])
replaced_data[5] = pd.DataFrame(replaced_data[5].iloc[:,0])
replaced_data[6] = pd.DataFrame(replaced_data[6].iloc[:,0])
replaced_data[7] = pd.DataFrame(replaced_data[7].iloc[:,0] + replaced_data[7].iloc[:,1] + replaced_data[7].iloc[:,2] + replaced_data[7].iloc[:,3])
replaced_data[8] = pd.DataFrame(replaced_data[8].iloc[:,0] + replaced_data[8].iloc[:,1] + replaced_data[8].iloc[:,2])
replaced_data[9] = pd.DataFrame(replaced_data[9].iloc[:,0] + replaced_data[9].iloc[:,1] + replaced_data[9].iloc[:,2] + replaced_data[9].iloc[:,3] + replaced_data[9].iloc[:,4])
replaced_data[10] = pd.DataFrame(replaced_data[10].iloc[:,0] + replaced_data[10].iloc[:,1] + replaced_data[10].iloc[:,2] + replaced_data[10].iloc[:,3] + replaced_data[10].iloc[:,4])
replaced_data[11] = pd.DataFrame(replaced_data[11].iloc[:,0] + replaced_data[11].iloc[:,1] + replaced_data[11].iloc[:,2] + replaced_data[11].iloc[:,3] + replaced_data[11].iloc[:,4])
replaced_data[12] = pd.DataFrame(replaced_data[12].iloc[:,0])

Writing the Column names

In [59]:
counter = 0
for i in range(1, len(replaced_data)+1):
    column_header = 'Q' + str(i)
    replaced_data[counter].columns = [column_header]
    counter += 1

   

Final Updated Dataset Making

In [60]:
updated_dataframe = pd.DataFrame()
for i in replaced_data:
  updated_dataframe = pd.concat([updated_dataframe,i],axis=1)
updated_dataframe = pd.concat([df_1, updated_dataframe], axis=1)



In [61]:
updated_dataframe.to_csv("formated.csv")


# Part 2
Missing Values and their Reasons

In [62]:
data = pd.read_csv('formated.csv')


Missing columns Identification 

In [63]:
missing_columns = data.columns[data.isna().any()].tolist()
missing_columns

['Access Code',
 'Email Address',
 'Region',
 'City / Town / District',
 'Q4',
 'Q5',
 'Q6',
 'Q7',
 'Q8',
 'Q9',
 'Q10',
 'Q11',
 'Q12',
 'Q13']

There are two columns which are entirely zero so there is no point of cleaning it so dropping those two columns

In [65]:
data = data.drop(["Access Code", "Email Address"], axis=1)



The region column has alot of repeated values so they have to cleaned with the most occurence value

In [66]:
repeated_region = data["Region"].mode()
data["Region"].fillna(repeated_region.values[0], inplace=True)
missing_column = data.columns[data.isna().any()].tolist()
missing_column


['City / Town / District',
 'Q4',
 'Q5',
 'Q6',
 'Q7',
 'Q8',
 'Q9',
 'Q10',
 'Q11',
 'Q12',
 'Q13']

Same method is applied to City / Town / District Column as well

In [69]:
repeated_region = data["City / Town / District"].mode()
data["City / Town / District"].fillna(repeated_region.values[0], inplace=True)
missing_column = data.columns[data.isna().any()].tolist()
missing_column


['Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13']

Q3 which is Age column has two breakdowns. one is a space between data and other is age which is written is 1993. 
The obvious cleaning for this is if age is more than 100 then it will automatically subtract it from 2023 which is current year

In [71]:
import numpy as np


data['Q3'] = data['Q3'].str.replace(' ', '')
# as the values are in string so first we need to convert them into integer
data["Q3"] = data['Q3'].astype(int)
data['Q3'] = np.where(data['Q3'] > 100, 2022 - data['Q3'], data['Q3'])
data['Q3']


0     23
1     24
2     40
3     26
4     30
5     25
6     24
7     30
8     24
9     26
10    24
11    25
12    25
13    23
14    31
15    26
16    40
17    23
18    25
19    22
20    23
21    25
22    22
23    23
24    27
25    28
26    23
27    35
28    27
29    25
30    55
31    23
32    29
33    23
34    24
35    28
Name: Q3, dtype: int32

As Age is a numerical data type so its better to use median in it as it is quantative data type but there are two anomilies 6I and 7O which is creating problems so first have to correct it. 

In [75]:
data['Q4'] = np.where(data['Q4'] == "7O", "70", data['Q4'])
data['Q4'] = np.where(data['Q4'] == "6l", "61", data['Q4'])

# now converting the entire column to float
data['Q4'] = data['Q4'].astype(float)

median_region = data["Q4"].mean()
data["Q4"].fillna(median_region, inplace=True)
missing_column = data.columns[data.isna().any()].tolist()
missing_column
data['Q4']


0     63.0
1     59.0
2     70.0
3     66.0
4     70.0
5     67.0
6     70.0
7      5.2
8     71.0
9     71.0
10    66.0
11    66.0
12    69.0
13    66.0
14    67.0
15    72.0
16    68.0
17    60.0
18    67.0
19    67.0
20    74.0
21    65.0
22    67.0
23    75.0
24    67.0
25    68.0
26    72.0
27    61.0
28    64.0
29    71.0
30    56.0
31    70.0
32    70.0
33    73.0
34    68.0
35    66.0
Name: Q4, dtype: float64

Q5 is a country data so taking mode of it for missing values. Then there are some instances where there is alphabets in the lower and there are some cases where USA is written United States of America etc. 

In [78]:
repeated_region = data["Q5"].mode()
data["Q5"].fillna(repeated_region.values[0], inplace=True)
data['Q5'] = data['Q5'].str.strip()
data['Q5'] = data['Q5'].str.upper()
data['Q5'] = np.where(data['Q5'] == 'UNITED STATES ; PAKISTAN',
                     'UNITED STATES', data['Q5'])

data['Q5']


0            TAIWAN
1             INDIA
2             INDIA
3       SOUTH KOREA
4                US
5             INDIA
6          PAKISTAN
7             INDIA
8             CHINA
9        UZBEKISTAN
10              USA
11            INDIA
12          ROMANIA
13            INDIA
14            CHINA
15       UZBEKISTAN
16              USA
17            INDIA
18         ETHIOPIA
19    UNITED STATES
20    UNITED STATES
21            INDIA
22              USA
23          NIGERIA
24              USA
25           TAIWAN
26            INDIA
27          NIGERIA
28            INDIA
29            INDIA
30    UNITED STATES
31            INDIA
32    UNITED STATES
33              USA
34          VIETNAM
35       UZBEKISTAN
Name: Q5, dtype: object

Q6 is undergraduate degree so first I took the mode and then filled the missing value. After this there was one value YES which had to be replaced with NaN as there was no correspondence value for that. 

In [80]:
repeated_region = data["Q6"].mode()
data["Q6"].fillna(repeated_region.values[0], inplace=True)
data['Q6'] = data['Q6'].str.strip()
data['Q6'] = data['Q6'].str.upper()
data['Q6'] = data['Q6'].replace('YES',np.NaN)
data['Q6']

0                    ECONOMICS
1                      FINANCE
2        COMPUTER APPLICATIONS
3          BUSINESS MANAGEMENT
4                    ECONOMICS
5                    ECONOMICS
6                      FINANCE
7                    ECONOMICS
8                     ECONOMIC
9       ECONOMICS WITH FINANCE
10                         BIT
11        COMPUTER ENGINEERING
12                     BIOLOGY
13                         CSE
14                  MANAGEMENT
15      ECONOMICS WITH FINANCE
16                   ECONOMICS
17                 ENGINEERING
18         ACCOUNTING AND ISOM
19                  STATISTICS
20            COMPUTER SCIENCE
21            COMPUTER SCIENCE
22    MATHEMATICS & STATISTICS
23                   ECONOMICS
24         SYSTEMS ENGINEERING
25           CIVIL ENGINEERING
26                         ECE
27          QUANTITY SURVEYING
28     BACHELOR OF ENGINEERING
29                B.TECH IN CS
30               BS ACCOUNTING
31     BACHELOR OF ENGINEERING
32    CR

Q7 was the date paramter which is a string so took the repeated region method

In [81]:
repeated_region = data["Q7"].mode()
data["Q7"].fillna(repeated_region.values[0], inplace=True)


Q8 was the column which had the type of machine user has which has anomilies within the string provided. So a common string was indentified as than replaced. for missing values the mode technique was used

In [85]:
data['Q8'] = np.where(data['Q8'] == '2', 'Apple/MacBook', data['Q8'])
data['Q8'] = np.where(data['Q8'] == 'OtherAcer/Windows',
                     'Microsoft/Windows', data['Q8'])
data['Q8'] = np.where(
    data['Q8'] == 'Microsoft/WindowsApple/MacBook', 'Microsoft/Windows', data['Q8'])
data['Q8'] = np.where(data['Q8'] == '0', 'Microsoft/Windows', data['Q8'])

repeated_region = data["Q8"].mode()
data["Q8"].fillna(repeated_region.values[0], inplace=True)
data['Q8']


0         Apple/MacBook
1         Apple/MacBook
2         Apple/MacBook
3     Microsoft/Windows
4         Apple/MacBook
5     Microsoft/Windows
6     Microsoft/Windows
7     Microsoft/Windows
8     Microsoft/Windows
9     Microsoft/Windows
10    Microsoft/Windows
11    Microsoft/Windows
12        Apple/MacBook
13    Microsoft/Windows
14    Microsoft/Windows
15    Microsoft/Windows
16    Microsoft/Windows
17    Microsoft/Windows
18    Microsoft/Windows
19        Apple/MacBook
20    Microsoft/Windows
21    Microsoft/Windows
22    Microsoft/Windows
23    Microsoft/Windows
24    Microsoft/Windows
25    Microsoft/Windows
26        Apple/MacBook
27    Microsoft/Windows
28    Microsoft/Windows
29        Apple/MacBook
30    Microsoft/Windows
31    Microsoft/Windows
32    Microsoft/Windows
33    Microsoft/Windows
34    Microsoft/Windows
35    Microsoft/Windows
Name: Q8, dtype: object

Q9 again has only missing values so mode technique sorted it out

In [86]:
repeated_region = data["Q9"].mode()
data["Q9"].fillna(repeated_region.values[0], inplace=True)
data["Q9"]

0     Not Working while attending Mason
1            Working, but not Full Time
2                        Yes, Full Time
3     Not Working while attending Mason
4                        Yes, Full Time
5     Not Working while attending Mason
6            Working, but not Full Time
7     Not Working while attending Mason
8                        Yes, Full Time
9     Not Working while attending Mason
10    Not Working while attending Mason
11    Not Working while attending Mason
12           Working, but not Full Time
13    Not Working while attending Mason
14    Not Working while attending Mason
15    Not Working while attending Mason
16                       Yes, Full Time
17    Not Working while attending Mason
18    Not Working while attending Mason
19           Working, but not Full Time
20                       Yes, Full Time
21    Not Working while attending Mason
22           Working, but not Full Time
23    Not Working while attending Mason
24                       Yes, Full Time


In [87]:
repeated_region = data["Q10"].mode()
data["Q10"].fillna(repeated_region.values[0], inplace=True)
data["Q10"]


0              Some familiarity
1                  Average user
2              Some familiarity
3                  Average user
4              Some familiarity
5                  Average user
6                  Average user
7                  Average user
8              Some familiarity
9                  Average user
10                  Little/none
11    Frequent use for projects
12             Some familiarity
13                 Average user
14                 Average user
15             Some familiarity
16                 Average user
17    Frequent use for projects
18                  Little/none
19             Some familiarity
20             Some familiarity
21             Some familiarity
22             Some familiarity
23                  Little/none
24                 Average user
25                 Average user
26                 Average user
27                 Average user
28                 Average user
29                 Average user
30                  Little/none
31      

In [88]:
repeated_region = data["Q11"].mode()
data["Q11"].fillna(repeated_region.values[0], inplace=True)
data["Q11"]


0                   Little/none
1                  Average user
2                   Little/none
3     Frequent use for projects
4              Some familiarity
5                  Average user
6                  Average user
7                  Average user
8     Frequent use for projects
9                  Average user
10                  Little/none
11                 Average user
12                 Average user
13                 Average user
14                  Little/none
15             Some familiarity
16             Some familiarity
17    Frequent use for projects
18                  Little/none
19                Fluent/expert
20             Some familiarity
21                  Little/none
22    Frequent use for projects
23                  Little/none
24                 Average user
25             Some familiarity
26    Frequent use for projects
27    Frequent use for projects
28                 Average user
29                 Average user
30             Some familiarity
31    Fr

In [89]:
repeated_region = data["Q12"].mode()
data["Q12"].fillna(repeated_region.values[0], inplace=True)
data["Q12"]


0              Some familiarity
1                   Little/none
2                  Average user
3              Some familiarity
4                  Average user
5                  Average user
6                  Average user
7                  Average user
8                  Average user
9                  Average user
10             Some familiarity
11    Frequent use for projects
12                 Average user
13                 Average user
14                  Little/none
15             Some familiarity
16             Some familiarity
17                 Average user
18                 Average user
19                 Average user
20             Some familiarity
21                 Average user
22                  Little/none
23                  Little/none
24                 Average user
25             Some familiarity
26                 Average user
27                 Average user
28                 Average user
29    Frequent use for projects
30             Some familiarity
31    Fr

Q13 has long sentences so I just filled it with missing values

In [90]:
data["Q13"].fillna('', inplace=True)


Transfering the file to formatted

In [91]:
data.to_csv("cleaned.csv")


# Part 3
Statistical Analysis

In [92]:
counter = 0
statistical_data = []
for i in range(1, len(replaced_data)+1):
    column_header = 'Q' + str(i)
    temp = data[column_header].describe()
    statistical_data.append(temp)
    counter += 1


In [93]:
statistical_data[0]

count    36.000000
mean      2.888889
std       1.007905
min       2.000000
25%       2.000000
50%       2.000000
75%       4.000000
max       4.000000
Name: Q1, dtype: float64

Result: Mean is close to 3 so it means that most of the students belong to 003 section

In [94]:
statistical_data[1]


count       36
unique       4
top       Male
freq        18
Name: Q2, dtype: object

Result: This shows that majority of the students are Male

In [95]:
statistical_data[2]


count    36.000000
mean     27.111111
std       6.492975
min      22.000000
25%      23.000000
50%      25.000000
75%      28.000000
max      55.000000
Name: Q3, dtype: float64

Result: The average age of student enrolled is 27 however there are range of students from 22 to 55 enrolled which creates a diversity in class. 

In [96]:
statistical_data[3]


count    36.000000
mean     65.755556
std      11.169459
min       5.200000
25%      66.000000
50%      67.000000
75%      70.000000
max      75.000000
Name: Q4, dtype: float64

Result: On average the height of student is 65. the standard deviation shows that their is widespread of diversity which shows that how far the height is spreaded. 

In [97]:
statistical_data[4]


count        36
unique       13
top       INDIA
freq         12
Name: Q5, dtype: object

Result: There are 13 unique students belonging from unique ethnicity however majority of students are from INDIA

In [98]:
statistical_data[5]


count            36
unique           25
top       ECONOMICS
freq              7
Name: Q6, dtype: object

Result: This shows that Economic is the favorite subject in wwhich most of the students are enrolled in. 

In [99]:
statistical_data[6]


count             36
unique            24
top       2021-12-20
freq               6
Name: Q7, dtype: object

Result: It shows that students will be graduating in 2021-12-20

In [100]:
statistical_data[7]


count                    36
unique                    2
top       Microsoft/Windows
freq                     28
Name: Q8, dtype: object

Result: It seems that Majority of the students use Microsoft Windows Laptop

In [101]:
statistical_data[8]


count                                    36
unique                                    3
top       Not Working while attending Mason
freq                                     20
Name: Q9, dtype: object

Result: It seems that majority of the students dont work while being enrolled in Mason

In [102]:
statistical_data[9]


count               36
unique               4
top       Average user
freq                18
Name: Q10, dtype: object

Result: Around half of the students are considered as an average coder in Python

In [103]:
statistical_data[10]


count               36
unique               5
top       Average user
freq                12
Name: Q11, dtype: object

Result: many students consider themselves as an average coder in R.

In [104]:
statistical_data[11]


count               36
unique               4
top       Average user
freq                18
Name: Q12, dtype: object

Result: Similar Intrepreation as above

In [105]:
statistical_data[12]


count     36
unique    28
top         
freq       9
Name: Q13, dtype: object